In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections
import gridded
import xarray as xr
import datetime

from osgeo import gdal, osr
%matplotlib inline

In [2]:
def create_tif(destName, array, fill_value, rows, cols, top_left_x, top_left_y, x_step=0.1, y_step=0.1, rotation=0):
    """
    function to create tif file
    """
    
    # Create output tiff file.
    # driver.Create() parameters are: output path, number of columns, number of rows,
    # number of bands, data type
    driver = gdal.GetDriverByName("GTiff")
    out_tif = driver.Create(destName, cols, rows, 1, gdal.GDT_Float32)

    # Create Spatial Reference object and set GeoTIFF projection.
    # This information may be found in either the data documentation or the netCDF file.
    prj = osr.SpatialReference()
    prj.ImportFromEPSG(28992) # WGS84
    out_tif.SetProjection(prj.ExportToWkt())

    # Set GeoTransformation.
    # This information may be found in either the data documentation, the netCDF file, or
    # can be derived. For example, if you know the longitude range and number of columns
    # you can calculate the x step as float(lon_range)/float(num_cols).
    geotrans = [top_left_x,x_step,rotation,top_left_y,rotation,-y_step]
    out_tif.SetGeoTransform(geotrans)

    # Finally we can write the array to the raster band.
    out_band = out_tif.GetRasterBand(1)    
    out_band.SetNoDataValue(fill_value)
    out_band.WriteArray(array)

    # Clear the memory and close the output file.
    out_tif.FlushCache()
    out_tif = None

In [6]:
file_nc = (r't:\2mattijn\c01_harlingen\dflowfm\dflowfmoutput\001_map.nc')
ds = gridded.Dataset(file_nc)

ValueError: Node coordinates standard_name is neither "longitude" nor "latitude" 

In [8]:
gridded.Dataset(file_nc, variables={"longitude": "mesh2d_node_x",
                                    "latitude":"mesh2_nodex_y"
                                   })

ValueError: You can create a Dataset from a file, or from raw databut not both.

In [6]:
# what variables are there"
print("Variables available and their standard names:\n")
for name, var in ds.variables.items():
    try:
        print(name, ":", var.attributes['standard_name'])
    except KeyError:
        pass

Variables available and their standard names:



AttributeError: 'Grid_R' object has no attribute 'variables'

In [5]:
ds = xr.open_dataset(url)
ds

<xarray.Dataset>
Dimensions:                      (Two: 2, max_nmesh2d_face_nodes: 5, nmesh2d_edge: 29441, nmesh2d_face: 16597, nmesh2d_node: 12845, time: 2)
Coordinates:
    mesh2d_node_x                (nmesh2d_node) float64 ...
    mesh2d_node_y                (nmesh2d_node) float64 ...
    mesh2d_edge_x                (nmesh2d_edge) float64 ...
    mesh2d_edge_y                (nmesh2d_edge) float64 ...
    mesh2d_face_x                (nmesh2d_face) float64 ...
    mesh2d_face_y                (nmesh2d_face) float64 ...
  * time                         (time) datetime64[ns] 2008-01-19T12:00:00 ...
Dimensions without coordinates: Two, max_nmesh2d_face_nodes, nmesh2d_edge, nmesh2d_face, nmesh2d_node
Data variables:
    mesh2d                       int32 ...
    projected_coordinate_system  int32 ...
    mesh2d_node_z                (nmesh2d_node) float64 ...
    mesh2d_edge_nodes            (nmesh2d_edge, Two) float64 ...
    mesh2d_edge_x_bnd            (nmesh2d_edge, Two) float64 

In [ ]:
sec_arr = np.array(ds.timestamp[:])

In [ ]:
[datetime.datetime.fromtimestamp(sec) for sec in sec_arr]

In [ ]:
xgrid = np.array(ds.xgrid[0])

In [ ]:
xgrid

In [ ]:
ygrid_ = np.array(ds.ygrid[:])

In [ ]:
ygrid = ygrid_[:,0]

In [ ]:
ygrid.shape, xgrid.shape

In [ ]:
# ygrid = np.array(ds.ygrid[:,0])

In [ ]:
y, x = np.meshgrid(ygrid, xgrid)

In [ ]:
# ds.dune_beach.attrs

In [ ]:
z = np.array(ds.zgrid_interpolated[0])

In [ ]:
z.shape

In [ ]:
fill_value = z[0][0]

In [ ]:
plt.imshow(z)
plt.show()

In [ ]:
xgrid

In [ ]:
ygrid

In [ ]:
fig, ax = plt.subplots()
graph = ax.pcolorfast(xgrid, ygrid, z, cmap='viridis')
plt.colorbar(graph)
plt.show()

In [ ]:
create_tif(destName=r'D:\Projects\RO\netcdf_WB//zgrid_interpolated_2015050240200_v2.tif', array=z,
           fill_value=fill_value.astype(float), rows=ygrid.shape[0], cols=xgrid.shape[0], top_left_x=xgrid[0], 
           top_left_y=ygrid[0], x_step=0.5, y_step=0.5)